# Лабораторна робота №5
# Дровольського Ярослава, ПЗС 1 курс магістратури

**Завдання:** Підібрати оптимальні гіперпараметри до нейронної мережі (багатошарового персептрону) з lab4.

*Спочатку нагадаємо зміст задачі та суть моделі*

Задача моделі: задача класифікації одягу на датасеті Fashion MNIST.

Всі дані (зображення одягу) в Fashion MNIST поділяються на 10 класів:

0. T-shirt / top (футболка / топ)
1. Trouser (брюки)
2. Pullover (пуловер)
3. Dress (плаття)
4. Coat (пальто)
5. Sandal (сандалі)
6. Shirt (сорочка)
7. Sneaker (кеди)
8. Bag (сумка)
9. Ankle boot (ботильйони).

- **Вхідні значення нейронної мережі:** інтенсивність пікселя (число в діапазоні 0-255), центрована і нормована до інтервалу [-0.5, 0.5]. В зображені 784 (28*28) пікселів. Вхідний шар 784 нейрона.
- **Вихідний шар** 10 нейронів. Кожен нейрон повертає ймовірність (0-1) того, що на зображенні предмет одягу з певної категорії (10 категорій = 10 нейронів)

## Підготовка даних

Імпортуємо необхідні бібліотеки

In [89]:
import numpy as np
import sklearn # algorithms for classical ML
import tensorflow as tf # build and train Neural network
from tensorflow import keras

Завантажимо датасет Fashion MNIST

In [90]:
from keras.datasets import mnist

(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

# x_train, x_val - clothes images (28x28 px) for train and validation sample, correspondingly (sample - вибірка)
# y_train, y_val - correct answers for corresponding images

Центруємо і нормуємо вхідні дані, так, щоб значення змінювалися від `-0.5` до `+0.5`.

In [91]:
x_train_float = x_train.astype(np.float64) / 255 - 0.5
x_val_float = x_val.astype(np.float64) / 255 - 0.5

Перетворимо правильні відповіді `y_train` і `y_val` в one-hot encode.

Тобто у кожного об'єкта з `y_train` та `y_val` створюється 10 нових ознак (кожна відповідає за те, чи належать об'єкт певній категорії). Ознака, яка відповідає за ту категорію, якій належить об'єкт, дорівнює `1`, решта дорівнюють `0`.

In [92]:
y_train_oh = keras.utils.to_categorical(y_train, 10)
y_val_oh = keras.utils.to_categorical(y_val, 10)

## Робота з моделлю

Згідно зі слайдом №6 презентації, **Схема** нашої подальшої роботи для кожного завдання така:
1. перебираємо всі дані в умові значення гіперпараметру
2. при цьому робимо крос-валідацію (перересну перевірку)
3. на основі результатів перехресної перевірки обираємо найкраще значення гіперпараметру

### Базова модель

Для отримання однакових похибок при повторному запуску клітинок навчання моделі (наприклад, під час перевірки викладачем), **фіксуємо** `seed` **рандомності**.


*Пояснення: When re-training a Keras neural network on the same data as before, you’ll rarely get the same results twice. This is due to the fact that neural networks in Keras are using randomness when initializing their weights, so on every run weights are initialized differently, therefore during the learning process these will get updated differently, so the same accuracy results when making predictions are unlikely.* [(Джерело)](https://medium.com/@pop.kristina1/why-loading-a-previously-saved-keras-model-gives-different-results-lessons-learned-aeea1014e0ba)

In [93]:
np.random.seed(100)

import tensorflow
tensorflow.random.set_seed(100)

**Тепер перейдемо до складання моделі й підбору її параметрів**.

На **вхід** будемо подавати картинки, витягнуті у вектор довжини 28 * 28 (= 784).

На **виході** маємо 10 вихідних нейронів за кількістю класів в нашій задачі.

Задаємо ці та описані вище параметри.


Побудуємо **БАЗОВУ МОДЕЛЬ** багатошарового персептрона з двома прихованими шарами по 128 нейронів у кожному. Саме для цієї моделі будемо шукати оптимальні значення параметрів.

На прихованих шарах використовуватимемо функцію elu.

Створимо функцію для побудови базової моделі

In [108]:
from keras import backend as K
from keras import layers as L


def create_basic_model():
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(128, input_dim=784, activation='relu')) # relu(x)=max(0,x)
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics, configure for training
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy', 'RootMeanSquaredError'] # calculated using validation_data
  )

  return model

'''
How to train:

  # train the model
  model.fit(
    x_train_float.reshape(-1, 28*28),
    y_train_oh,
    batch_size=64, # Number of samples per gradient update.
    epochs=10,
    validation_data=(x_val_float.reshape(-1, 28*28), y_val_oh) # The model will not be trained on this data.
  )

'''


 # Dense is Just regular densely-connected NN layer.
 # first argument of Dense constructor is `units` - Positive integer, dimensionality of the output space.
 # Dense implements the operation: output = activation(dot(input, kernel) + bias)


  # Model documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
  # Layer documentation: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense


'\nHow to train:\n\n  # train the model\n  model.fit(\n    x_train_float.reshape(-1, 28*28),\n    y_train_oh,\n    batch_size=64, # Number of samples per gradient update.\n    epochs=10,\n    validation_data=(x_val_float.reshape(-1, 28*28), y_val_oh) # The model will not be trained on this data.\n  )\n\n'

Напишемо функцію для виконання **крос-валідації** моделі

In [95]:
# .fit() calls on already trained models, will train model from xero (ignore previous trained coeffircients, etc.) - not for Keras
# function does not call .fit() on passed model - but call on its copy - https://stackoverflow.com/a/49771618 - not for Keras

from sklearn.model_selection import KFold
import time

# create_model_function - is function to create model. It should return ONLY created and compiled model, NOT trained.
# calculates accuracies for cross-validation
# (X, y) are TRAIN data. It will be divided into local-train and local-validation data on each fold.
# X element is 28x28 images (matrices with elements from interval [-0.5, 0.5]). y element is 10-elements vector
def cross_validate(X, y, create_model_function, batch_size, number_of_epochs):
  kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds

  fold_losses = []
  fold_accuracies = []
  fold_rmse = []
  train_times = []

  for train_index, val_index in kf.split(X):
      X_train, X_val = X[train_index], X[val_index]
      y_train, y_val = y[train_index], y[val_index]

      model = create_model_function()  # Create a new model instance, Keras models need to be re-initialized for each fold.

      time_start = time.time()
      model.fit(
          X_train.reshape(-1, 28*28),
          y_train,
          epochs=number_of_epochs,
          batch_size=batch_size,
          verbose=0)  # Train the model
      time_end = time.time()

      loss, accuracy, rmse = model.evaluate(X_val.reshape(-1, 28*28), y_val, batch_size=batch_size, verbose=1)  # Evaluate on local-validation set

      fold_losses.append(loss)
      fold_accuracies.append(accuracy)
      fold_rmse.append(rmse)
      train_times.append(time_end - time_start) # time in seconds

  return fold_losses, fold_accuracies, fold_rmse, train_times

In [80]:
import statistics

def display_cross_validation_results(losses, accuracies, rmses, train_times):
  print("\n")
  print("Losses: ", losses)
  print("Mean loss", statistics.mean(losses))
  print("Standard deviation of losses:", statistics.stdev(losses))

  print("")
  print("Accuracies: ", accuracies)
  print("Mean accuracy", statistics.mean(accuracies))
  print("Standard deviation of accuracy:", statistics.stdev(accuracies))

  print("")
  print("RMSEs: ", rmses)
  print("Mean RMSE", statistics.mean(rmses))
  print("Standard deviation of RMSE:", statistics.stdev(rmses))

  print("")
  print("Mean train time: ", statistics.mean(train_times), " seconds")

### №1 Підбір кількості нейронів на вхідному шарі
Використайте різну кількість нейронів на вхідному шарі: 400, 600, 800, 1200. Аргументуйте відповідь.

Для цього створимо функцію, яка буде створювати модель, ідентичну з базовою, але яка *має різну кількість нейронів на вхідному шарі*.

In [78]:
def create_basic_model_with_input_layer_neurons(input_layer_neurons_number):
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(input_layer_neurons_number, input_dim=784, activation='relu'))
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy', 'RootMeanSquaredError'] # calculated using validation_data
  )

  return model

In [66]:
def task_1(number_of_neurons):
  losses, accuracies, rmses, train_times = cross_validate(
    x_train_float, y_train_oh,
    lambda: create_basic_model_with_input_layer_neurons(number_of_neurons),
    batch_size=64,
    number_of_epochs=10)

  display_cross_validation_results(losses, accuracies, rmses, train_times)

**1)** 400 нейронів на вхідному шарі

In [79]:
task_1(400)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - RootMeanSquaredError: 0.1339 - accuracy: 0.8777 - loss: 0.3683
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1341 - accuracy: 0.8796 - loss: 0.3761
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1352 - accuracy: 0.8765 - loss: 0.3837
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1283 - accuracy: 0.8862 - loss: 0.3440
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1356 - accuracy: 0.8753 - loss: 0.3802


Losses:  [0.3679735064506531, 0.3775351643562317, 0.37524646520614624, 0.35480597615242004, 0.3864053189754486]
Mean loss 0.3723932862281799
Standard deviation of losses: 0.011829671716253756

Accuracies:  [0.878166675567627, 0.8811666369438171, 0.8805833458900452, 0.8830000162124634, 0.8765833377838135]
Mean accuracy 0.8799000024795532
Standard deviation of accuracy: 0.0025340724690964215

RMSEs:  [0.13368059694766998, 0.1335497945547104, 0.13342531025409698, 0.1

**2)** 600 нейронів на вхідному шарі

In [81]:
task_1(600)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1331 - accuracy: 0.8805 - loss: 0.3658
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1360 - accuracy: 0.8787 - loss: 0.4101
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1358 - accuracy: 0.8780 - loss: 0.3933
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1305 - accuracy: 0.8841 - loss: 0.3724
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1313 - accuracy: 0.8840 - loss: 0.3639


Losses:  [0.3717436194419861, 0.4130309224128723, 0.381742924451828, 0.37960347533226013, 0.3712914288043976]
Mean loss 0.3834824740886688
Standard deviation of losses: 0.017158130406592045

Accuracies:  [0.8770833611488342, 0.8769999742507935, 0.8813333511352539, 0.8799166679382324, 0.8820833563804626]
Mean accuracy 0.8794833421707153
Standard deviation of accuracy: 0.0023610063220818513

RMSEs:  [0.13415025174617767, 0.13637326657772064, 0.13410817086696625, 0.1

**3)** 800 нейронів на вхідному шарі

In [85]:
task_1(800)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1349 - accuracy: 0.8763 - loss: 0.3898
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - RootMeanSquaredError: 0.1396 - accuracy: 0.8697 - loss: 0.4198
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1334 - accuracy: 0.8823 - loss: 0.3781
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1320 - accuracy: 0.8821 - loss: 0.3685
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - RootMeanSquaredError: 0.1294 - accuracy: 0.8833 - loss: 0.3546


Losses:  [0.38826894760131836, 0.4224608540534973, 0.36891844868659973, 0.3763483464717865, 0.3622235655784607]
Mean loss 0.3836440324783325
Standard deviation of losses: 0.023754166643006874

Accuracies:  [0.8733333349227905, 0.871666669845581, 0.8868333101272583, 0.8804166913032532, 0.8820833563804626]
Mean accuracy 0.8788666725158691
Standard deviation of accuracy: 0.006298309128812126

RMSEs:  [0.13561083376407623, 0.13907648622989655, 0.1311388909816742, 0.13

**4)** 1200 нейронів на вхідному шарі

In [86]:
task_1(1200)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - RootMeanSquaredError: 0.1363 - accuracy: 0.8746 - loss: 0.3855
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - RootMeanSquaredError: 0.1354 - accuracy: 0.8786 - loss: 0.3916
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - RootMeanSquaredError: 0.1388 - accuracy: 0.8719 - loss: 0.4172
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - RootMeanSquaredError: 0.1309 - accuracy: 0.8820 - loss: 0.3545
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - RootMeanSquaredError: 0.1348 - accuracy: 0.8764 - loss: 0.3867


Losses:  [0.38206619024276733, 0.4070315659046173, 0.40473634004592896, 0.3662453591823578, 0.39513298869132996]
Mean loss 0.39104248881340026
Standard deviation of losses: 0.01699541049097317

Accuracies:  [0.875249981880188, 0.8773333430290222, 0.8767499923706055, 0.8801666498184204, 0.8754166960716248]
Mean accuracy 0.8769833326339722
Standard deviation of accuracy: 0.001985530042966016

RMSEs:  [0.13543958961963654, 0.13657963275909424, 0.13605500757694244, 0.

**Висновок:** бачимо, що зі збільшенням кількості нейронів на вхідному шарі, точність моделі та інші обчислені вище показники майже не змінюються і коливаються в межах похибки. Причому, час навчання помітно збільшується (з 79 секунд до 190 секунд). Тому, в якості найкращої кількості нейронів на вхідному шарі оберемо найменший із запропонованих варіантів - **400**.

*Зауважимо, що ефект сталості точності моделі при збільшенні кількості нейронів  може виникати через: неякісні або надмірні дані (повторення, чи шуми), недостатню кількість даних, недостатню кількість прихованих шарів, неправильний вибір функції активації, неправильну функція оптимізації.*

### №2 Підбір кількості нейронів на першому прихованому шарі
Додайте в нейронну мережу прихований шар з різною кількістю нейронів: 200, 300, 400, 600, 800. Аргументуйте відповідь.

Для цього створимо функцію, яка буде створювати модель, ідентичну з базовою, але яка *має різну кількість нейронів на першому прихованому шарі*.

In [67]:
def create_basic_model_with_first_hide_layer_neurons(first_hide_layer_neurons_number):
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(128, input_dim=784, activation='relu'))
  model.add(L.Dense(first_hide_layer_neurons_number, activation='elu'))
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy', 'RootMeanSquaredError'] # calculated using validation_data
  )

  return model

In [68]:
def task_2(number_of_neurons):
  losses, accuracies, rmses, train_times = cross_validate(
    x_train_float, y_train_oh,
    lambda: create_basic_model_with_first_hide_layer_neurons(number_of_neurons),
    batch_size=64,
    number_of_epochs=10)

  display_cross_validation_results(losses, accuracies, rmses, train_times)

**1)** 200 нейронів на першому прихованому шарі

In [88]:
task_2(200)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1330 - accuracy: 0.8787 - loss: 0.3522
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1355 - accuracy: 0.8761 - loss: 0.3782
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1346 - accuracy: 0.8761 - loss: 0.3620
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1289 - accuracy: 0.8859 - loss: 0.3473
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1305 - accuracy: 0.8837 - loss: 0.3595


Losses:  [0.35090288519859314, 0.37181201577186584, 0.36185917258262634, 0.3524456322193146, 0.3734143376350403]
Mean loss 0.362086808681488
Standard deviation of losses: 0.010499723941416505

Accuracies:  [0.8794999718666077, 0.8769999742507935, 0.8785833120346069, 0.8835833072662354, 0.8795833587646484]
Mean accuracy 0.8796499848365784
Standard deviation of accuracy: 0.002431990708838165

RMSEs:  [0.13243025541305542, 0.13409525156021118, 0.13347095251083374, 0.

**2)** 300 нейронів на першому прихованому шарі

In [96]:
task_2(300)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1388 - accuracy: 0.8675 - loss: 0.3890
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1340 - accuracy: 0.8781 - loss: 0.3706
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1382 - accuracy: 0.8686 - loss: 0.3805
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1309 - accuracy: 0.8839 - loss: 0.3503
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1339 - accuracy: 0.8782 - loss: 0.3777


Losses:  [0.39004406332969666, 0.3719254434108734, 0.3700369894504547, 0.3646000921726227, 0.38884884119033813]
Mean loss 0.3770910859107971
Standard deviation of losses: 0.011602731328269675

Accuracies:  [0.8666666746139526, 0.8784999847412109, 0.875333309173584, 0.878333330154419, 0.8744166493415833]
Mean accuracy 0.87464998960495
Standard deviation of accuracy: 0.00481259160197145

RMSEs:  [0.13905790448188782, 0.13373740017414093, 0.13534504175186157, 0.13382

**3)** 400 нейронів на першому прихованому шарі

In [97]:
task_2(400)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1359 - accuracy: 0.8690 - loss: 0.3703
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1352 - accuracy: 0.8745 - loss: 0.3744
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1548 - accuracy: 0.8423 - loss: 0.5170
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - RootMeanSquaredError: 0.1290 - accuracy: 0.8886 - loss: 0.3492
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1317 - accuracy: 0.8797 - loss: 0.3431


Losses:  [0.36897972226142883, 0.37177926301956177, 0.5073665976524353, 0.35054755210876465, 0.35118716955184937]
Mean loss 0.389972060918808
Standard deviation of losses: 0.06635452721154723

Accuracies:  [0.8722500205039978, 0.8762500286102295, 0.846916675567627, 0.8854166865348816, 0.8765000104904175]
Mean accuracy 0.8714666843414307
Standard deviation of accuracy: 0.014544190094764948

RMSEs:  [0.13505730032920837, 0.13439837098121643, 0.15280669927597046, 0.1

**4)** 600 нейронів на першому прихованому шарі

In [98]:
task_2(600)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1331 - accuracy: 0.8784 - loss: 0.3510
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1360 - accuracy: 0.8731 - loss: 0.3892
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1367 - accuracy: 0.8747 - loss: 0.3884
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1319 - accuracy: 0.8809 - loss: 0.3645
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1350 - accuracy: 0.8780 - loss: 0.3863


Losses:  [0.35604938864707947, 0.38636547327041626, 0.38266322016716003, 0.36790934205055237, 0.38351261615753174]
Mean loss 0.37530000805854796
Standard deviation of losses: 0.012935577262591742

Accuracies:  [0.8725833296775818, 0.8759166598320007, 0.8794166445732117, 0.8765000104904175, 0.8755000233650208]
Mean accuracy 0.8759833335876465
Standard deviation of accuracy: 0.0024419582354588544

RMSEs:  [0.1339244693517685, 0.13507546484470367, 0.13473938405513763

**5)** 800 нейронів на першому прихованому шарі

In [99]:
task_2(800)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1349 - accuracy: 0.8742 - loss: 0.3601
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1392 - accuracy: 0.8693 - loss: 0.4002
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1364 - accuracy: 0.8711 - loss: 0.3869
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1297 - accuracy: 0.8834 - loss: 0.3442
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1324 - accuracy: 0.8811 - loss: 0.3640


Losses:  [0.3716716766357422, 0.3969607353210449, 0.3760949671268463, 0.3478260636329651, 0.38084012269973755]
Mean loss 0.3746787130832672
Standard deviation of losses: 0.017794411409767553

Accuracies:  [0.8725833296775818, 0.8711666464805603, 0.875083327293396, 0.8827499747276306, 0.8767499923706055]
Mean accuracy 0.8756666541099548
Standard deviation of accuracy: 0.004512324266787976

RMSEs:  [0.13599742949008942, 0.13819442689418793, 0.13494853675365448, 0.13

**Висновок:** бачимо, що зі збільшенням кількості нейронів на першому прихованому шарі, точність моделі та інші обчислені вище показники майже не змінюються і коливаються в межах похибки. Причому, час навчання помітно збільшується (з 57.8 секунд до майже 82 секунд). До того ж, при 200 нейронах (найменший запропонований варіант), точність найвища (87.96%). Тому, в якості найкращої кількості нейронів на першому прихованому шарі оберемо найменший із запропонованих варіантів - **200**.

*Зауважимо, що ефект сталості точності моделі при збільшенні кількості нейронів  може виникати через: неякісні або надмірні дані (повторення, чи шуми), недостатню кількість даних, недостатню кількість прихованих шарів, неправильний вибір функції активації, неправильну функція оптимізації.*

### №3 Підбір кількості нейронів на другому прихованому шарі
Використаємо різну кількість нейронів на другому прихованому шарі: 200, 300, 400, 600, 800.

Для цього створимо функцію, яка буде створювати модель, ідентичну з базовою, але яка *має різну кількість нейронів на другому прихованому шарі*.

In [83]:
def create_basic_model_with_second_hide_layer_neurons(second_hide_layer_neurons_number):
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(128, input_dim=784, activation='relu'))
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(second_hide_layer_neurons_number, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy', 'RootMeanSquaredError'] # calculated using validation_data
  )

  return model

In [84]:
def task_3(number_of_neurons):
  losses, accuracies, rmses, train_times = cross_validate(
    x_train_float, y_train_oh,
    lambda: create_basic_model_with_second_hide_layer_neurons(number_of_neurons),
    batch_size=64,
    number_of_epochs=10)

  display_cross_validation_results(losses, accuracies, rmses, train_times)

**1)** 200 нейронів на другому прихованому шарі

In [100]:
task_3(200)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1348 - accuracy: 0.8787 - loss: 0.3669
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1384 - accuracy: 0.8682 - loss: 0.4031
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1445 - accuracy: 0.8579 - loss: 0.4342
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1300 - accuracy: 0.8846 - loss: 0.3473
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1297 - accuracy: 0.8848 - loss: 0.3413


Losses:  [0.36697128415107727, 0.40091368556022644, 0.4288907051086426, 0.3573468029499054, 0.3559543788433075]
Mean loss 0.38201537132263186
Standard deviation of losses: 0.0318923241233898

Accuracies:  [0.8768333196640015, 0.8695833086967468, 0.8610000014305115, 0.8809166550636292, 0.8791666626930237]
Mean accuracy 0.8734999895095825
Standard deviation of accuracy: 0.008213088538933225

RMSEs:  [0.134813591837883, 0.13834504783153534, 0.1432795226573944, 0.1323

**2)** 300 нейронів на другому прихованому шарі

In [101]:
task_3(300)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1369 - accuracy: 0.8701 - loss: 0.3781
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1352 - accuracy: 0.8764 - loss: 0.3720
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1377 - accuracy: 0.8718 - loss: 0.3900
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1309 - accuracy: 0.8832 - loss: 0.3606
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1320 - accuracy: 0.8827 - loss: 0.3606


Losses:  [0.3841858208179474, 0.37108373641967773, 0.38875576853752136, 0.3660995364189148, 0.3633195757865906]
Mean loss 0.3746888875961304
Standard deviation of losses: 0.011226132456372434

Accuracies:  [0.8708333373069763, 0.878083348274231, 0.8739166855812073, 0.8825833201408386, 0.8788333535194397]
Mean accuracy 0.8768500089645386
Standard deviation of accuracy: 0.004558792669849293

RMSEs:  [0.13710100948810577, 0.13421113789081573, 0.13685214519500732, 0.1

**3)** 400 нейронів на другому прихованому шарі

In [102]:
task_3(400)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1410 - accuracy: 0.8663 - loss: 0.4093
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1334 - accuracy: 0.8796 - loss: 0.3661
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1382 - accuracy: 0.8699 - loss: 0.3842
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1288 - accuracy: 0.8825 - loss: 0.3462
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1315 - accuracy: 0.8784 - loss: 0.3435


Losses:  [0.41453272104263306, 0.3624531328678131, 0.3796321749687195, 0.3532926142215729, 0.35777905583381653]
Mean loss 0.373537939786911
Standard deviation of losses: 0.024994001119183606

Accuracies:  [0.8675833344459534, 0.8784999847412109, 0.8740833401679993, 0.8834166526794434, 0.875166654586792]
Mean accuracy 0.8757499933242798
Standard deviation of accuracy: 0.005834219153456902

RMSEs:  [0.14044025540351868, 0.13338571786880493, 0.13665175437927246, 0.13

**4)** 600 нейронів на другому прихованому шарі

In [103]:
task_3(600)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1387 - accuracy: 0.8661 - loss: 0.3936
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1441 - accuracy: 0.8594 - loss: 0.4285
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1374 - accuracy: 0.8691 - loss: 0.3801
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1311 - accuracy: 0.8801 - loss: 0.3451
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1323 - accuracy: 0.8791 - loss: 0.3575


Losses:  [0.3914164900779724, 0.4242706000804901, 0.381644606590271, 0.35822439193725586, 0.36318549513816833]
Mean loss 0.38374831676483157
Standard deviation of losses: 0.02636087202383069

Accuracies:  [0.8684166669845581, 0.8607500195503235, 0.8711666464805603, 0.8784166574478149, 0.8755833506584167]
Mean accuracy 0.8708666682243347
Standard deviation of accuracy: 0.006849972698506429

RMSEs:  [0.13792307674884796, 0.14279557764530182, 0.1367730349302292, 0.13

**5)** 800 нейронів на другому прихованому шарі

In [104]:
task_3(800)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1359 - accuracy: 0.8730 - loss: 0.3766
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1375 - accuracy: 0.8757 - loss: 0.3950
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1378 - accuracy: 0.8693 - loss: 0.3866
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1323 - accuracy: 0.8797 - loss: 0.3566
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1341 - accuracy: 0.8741 - loss: 0.3668


Losses:  [0.37837278842926025, 0.401824951171875, 0.375526487827301, 0.36286669969558716, 0.3708127439022064]
Mean loss 0.377880734205246
Standard deviation of losses: 0.014615811974481006

Accuracies:  [0.8705000281333923, 0.8734166622161865, 0.874833345413208, 0.8793333172798157, 0.8741666674613953]
Mean accuracy 0.8744500041007995
Standard deviation of accuracy: 0.003191554613780355

RMSEs:  [0.13624829053878784, 0.13793663680553436, 0.1355694830417633, 0.13327

**Висновок:** бачимо, що зі збільшенням кількості нейронів на другому прихованому шарі, точність моделі та інші обчислені вище показники майже не змінюються і коливаються в межах похибки. Причому, час навчання помітно збільшується (з 44.3 секунд до майже 69.09 секунд). Але при 300 нейронах, точність найвища (87.68%), хоча час навчання є несуттєво більшим за мінімальним час навчання (на 4 секунди). Тому, в якості найкращої кількості нейронів на другому прихованому шарі оберемо **300**.

*Зауважимо, що ефект сталості точності моделі при збільшенні кількості нейронів  може виникати через: неякісні або надмірні дані (повторення, чи шуми), недостатню кількість даних, недостатню кількість прихованих шарів, неправильний вибір функції активації, неправильну функція оптимізації.*

### №4 Підбір кількості епох тренування
Використовуйте різну кількість епох: 10, 15, 20, 25, 30. Аргументуйте відповідь.

Для підбору будемо використовувати базову модель.

In [106]:
def task_4(number_of_epochs):
  losses, accuracies, rmses, train_times = cross_validate(
    x_train_float, y_train_oh,
    lambda: create_basic_model(),
    batch_size=64,
    number_of_epochs=number_of_epochs)

  display_cross_validation_results(losses, accuracies, rmses, train_times)

**1)** 10 епох

In [109]:
task_4(10)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1340 - accuracy: 0.8774 - loss: 0.3589
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1338 - accuracy: 0.8767 - loss: 0.3627
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1359 - accuracy: 0.8773 - loss: 0.3734
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.1298 - accuracy: 0.8834 - loss: 0.3461
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1334 - accuracy: 0.8777 - loss: 0.3744


Losses:  [0.3635462522506714, 0.3650265038013458, 0.3669094145298004, 0.35467448830604553, 0.3891869783401489]
Mean loss 0.3678687274456024
Standard deviation of losses: 0.01280857209059462

Accuracies:  [0.8760833144187927, 0.8794999718666077, 0.8795833587646484, 0.8798333406448364, 0.875083327293396]
Mean accuracy 0.8780166625976562
Standard deviation of accuracy: 0.0022526292183838244

RMSEs:  [0.1342262476682663, 0.13293330371379852, 0.13396085798740387, 0.131

**2)** 15 епох

In [110]:
task_4(15)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1370 - accuracy: 0.8746 - loss: 0.4151
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1332 - accuracy: 0.8849 - loss: 0.4224
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1345 - accuracy: 0.8823 - loss: 0.3894
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1339 - accuracy: 0.8807 - loss: 0.4000
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1326 - accuracy: 0.8843 - loss: 0.3847


Losses:  [0.4136870503425598, 0.4179619550704956, 0.39180582761764526, 0.4027590751647949, 0.3937612771987915]
Mean loss 0.4039950370788574
Standard deviation of losses: 0.011660233592365263

Accuracies:  [0.874916672706604, 0.8838333487510681, 0.8819166421890259, 0.8794999718666077, 0.8802499771118164]
Mean accuracy 0.8800833225250244
Standard deviation of accuracy: 0.0033348950423734377

RMSEs:  [0.1369398981332779, 0.1332816183567047, 0.13421842455863953, 0.134

**3)** 20 епох

In [111]:
task_4(20)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1433 - accuracy: 0.8689 - loss: 0.5320
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1384 - accuracy: 0.8767 - loss: 0.4675
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1394 - accuracy: 0.8748 - loss: 0.4495
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1308 - accuracy: 0.8863 - loss: 0.4005
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1345 - accuracy: 0.8809 - loss: 0.4259


Losses:  [0.5374037623405457, 0.4518857002258301, 0.4579877257347107, 0.41266068816185, 0.43672648072242737]
Mean loss 0.45933287143707274
Standard deviation of losses: 0.047015091660261875

Accuracies:  [0.8685833215713501, 0.8804166913032532, 0.8759166598320007, 0.8840000033378601, 0.8798333406448364]
Mean accuracy 0.8777500033378601
Standard deviation of accuracy: 0.005871610129648327

RMSEs:  [0.14339196681976318, 0.1360986977815628, 0.13877151906490326, 0.132

**4)** 25 епох

In [112]:
task_4(25)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1483 - accuracy: 0.8610 - loss: 0.5909
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1328 - accuracy: 0.8883 - loss: 0.4876
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1399 - accuracy: 0.8744 - loss: 0.5132
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1367 - accuracy: 0.8805 - loss: 0.5059
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1356 - accuracy: 0.8827 - loss: 0.4804


Losses:  [0.6028618216514587, 0.4974357783794403, 0.5128278732299805, 0.5161992311477661, 0.4845253825187683]
Mean loss 0.5227700173854828
Standard deviation of losses: 0.046532768923828176

Accuracies:  [0.8612499833106995, 0.8848333358764648, 0.8756666779518127, 0.8795833587646484, 0.8810833096504211]
Mean accuracy 0.8764833331108093
Standard deviation of accuracy: 0.009127080053159355

RMSEs:  [0.14897309243679047, 0.13472051918506622, 0.13930386304855347, 0.13

**5)** 30 епох

In [113]:
task_4(30)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1415 - accuracy: 0.8741 - loss: 0.5589
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1392 - accuracy: 0.8806 - loss: 0.5791
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1418 - accuracy: 0.8753 - loss: 0.5916
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1356 - accuracy: 0.8838 - loss: 0.5535
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1413 - accuracy: 0.8759 - loss: 0.6130


Losses:  [0.5821890234947205, 0.5742327570915222, 0.5737327337265015, 0.5800963640213013, 0.6076533198356628]
Mean loss 0.5835808396339417
Standard deviation of losses: 0.013945741847485078

Accuracies:  [0.872083306312561, 0.8809999823570251, 0.8824999928474426, 0.8817499876022339, 0.8765000104904175]
Mean accuracy 0.878766655921936
Standard deviation of accuracy: 0.00440541323240637

RMSEs:  [0.1429816633462906, 0.13894684612751007, 0.13797259330749512, 0.137761

**Висновок:** бачимо, що точність досягає свого максимуму на 15 епохах. Далі, попри збільшення кількості епох, точність навчання залишається майже незмінною. Але суттєво збільшується loss (відстань між передбаченими та реальними значеннями): з 36.78 до 58.35. Окрім цього, збільшується час навчання: з 46.5 секунд до 138.68 секунд. Тому, в якості найкращої кількості епох оберемо **15**.

*До незбільшення точності зі збільшенням кількості епох можуть призвести  наступні чинники: перенавчання, сатурація (модель вже досягла максимальної продуктивності на даній задачі), занадто висока швидкість навчання (модель "перескакує" оптимальні значення, не досягнувши їх), якість даних (наприклад, наявність шумів), недостатня складність моделі.*

### №5 Підбір розміру міні-вибірки (batch_size)
Використовуйте різні розміри міні-вибірки (batch_size): 10, 50, 100, 200, 500 Аргументуйте відповідь.

Для підбору будемо використовувати базову модель.

In [115]:
def task_5(batch_size):
  losses, accuracies, rmses, train_times = cross_validate(
    x_train_float, y_train_oh,
    lambda: create_basic_model(),
    batch_size=batch_size,
    number_of_epochs=10)

  display_cross_validation_results(losses, accuracies, rmses, train_times)

**1)** batch_size = 10

In [116]:
task_5(10)

1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - RootMeanSquaredError: 0.1392 - accuracy: 0.8680 - loss: 0.4239
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - RootMeanSquaredError: 0.1388 - accuracy: 0.8678 - loss: 0.3887
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - RootMeanSquaredError: 0.1427 - accuracy: 0.8644 - loss: 0.4230
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 0.1352 - accuracy: 0.8763 - loss: 0.3815
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 0.1332 - accuracy: 0.8813 - loss: 0.4148


Losses:  [0.41938790678977966, 0.3883073627948761, 0.4124620854854584, 0.37656351923942566, 0.4254325032234192]
Mean loss 0.4044306755065918
Standard deviation of losses: 0.02101094174812936

Accuracies:  [0.8690000176429749, 0.8685833215713501, 0.8640833497047424, 0.8775833249092102, 0.8764166831970215]
Mean accuracy 0.8711333394050598
Standard deviation of accuracy: 0.005706838925692887

RMSEs:  [0.1389244794845581, 0.13835641741752625, 0.1412848383188

**2)** batch_size = 50

In [117]:
task_5(50)

240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1331 - accuracy: 0.8790 - loss: 0.3597
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1342 - accuracy: 0.8788 - loss: 0.3754
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1460 - accuracy: 0.8552 - loss: 0.4441
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.1310 - accuracy: 0.8832 - loss: 0.3541
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1315 - accuracy: 0.8810 - loss: 0.3591


Losses:  [0.36018380522727966, 0.3742499053478241, 0.4271431565284729, 0.35321861505508423, 0.3666364252567291]
Mean loss 0.376286381483078
Standard deviation of losses: 0.029474969331146413

Accuracies:  [0.8792499899864197, 0.8798333406448364, 0.8617500066757202, 0.8822500109672546, 0.8785833120346069]
Mean accuracy 0.8763333320617676
Standard deviation of accuracy: 0.008268915164565705

RMSEs:  [0.1331658512353897, 0.13348332047462463, 0.1432202160358429, 0.131

**3)** batch_size = 100

In [118]:
task_5(100)

120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1334 - accuracy: 0.8753 - loss: 0.3615
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1288 - accuracy: 0.8846 - loss: 0.3345
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.1371 - accuracy: 0.8712 - loss: 0.3776
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - RootMeanSquaredError: 0.1304 - accuracy: 0.8835 - loss: 0.3477
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.1281 - accuracy: 0.8871 - loss: 0.3358


Losses:  [0.37098416686058044, 0.33558934926986694, 0.37861841917037964, 0.3537523150444031, 0.3418694734573364]
Mean loss 0.3561627447605133
Standard deviation of losses: 0.01842080511437651

Accuracies:  [0.875333309173584, 0.8855833411216736, 0.8737499713897705, 0.8820000290870667, 0.8856666684150696]
Mean accuracy 0.8804666638374329
Standard deviation of accuracy: 0.005635533039754

RMSEs:  [0.13427908718585968, 0.12891662120819092, 0.1359967738389969, 0.13181

**4)** batch_size = 200

In [119]:
task_5(200)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - RootMeanSquaredError: 0.1381 - accuracy: 0.8690 - loss: 0.3734
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - RootMeanSquaredError: 0.1281 - accuracy: 0.8854 - loss: 0.3287
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - RootMeanSquaredError: 0.1329 - accuracy: 0.8769 - loss: 0.3485
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - RootMeanSquaredError: 0.1278 - accuracy: 0.8887 - loss: 0.3255
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - RootMeanSquaredError: 0.1310 - accuracy: 0.8804 - loss: 0.3332


Losses:  [0.37800541520118713, 0.3284386694431305, 0.3445371091365814, 0.3332425653934479, 0.34137555956840515]
Mean loss 0.3451198637485504
Standard deviation of losses: 0.019462472676580286

Accuracies:  [0.8661666512489319, 0.8855833411216736, 0.8798333406448364, 0.8833333253860474, 0.8768333196640015]
Mean accuracy 0.8783499956130981
Standard deviation of accuracy: 0.007584348376525225

RMSEs:  [0.13859763741493225, 0.12841898202896118, 0.13164401054382324, 0.1296599060

**5)** batch_size = 500

In [120]:
task_5(500)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - RootMeanSquaredError: 0.1286 - accuracy: 0.8842 - loss: 0.3191
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - RootMeanSquaredError: 0.1350 - accuracy: 0.8771 - loss: 0.3638
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - RootMeanSquaredError: 0.1307 - accuracy: 0.8806 - loss: 0.3267
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - RootMeanSquaredError: 0.1300 - accuracy: 0.8836 - loss: 0.3419
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - RootMeanSquaredError: 0.1291 - accuracy: 0.8844 - loss: 0.3212


Losses:  [0.3247508406639099, 0.36285820603370667, 0.3237805962562561, 0.3340245187282562, 0.3293701708316803]
Mean loss 0.33495686650276185
Standard deviation of losses: 0.01612170251990476

Accuracies:  [0.8825833201408386, 0.8743333220481873, 0.8809999823570251, 0.8836666941642761, 0.8820833563804626]
Mean accuracy 0.880733335018158
Standard deviation of accuracy: 0.0037040810375871926

RMSEs:  [0.12930484116077423, 0.13520169258117676, 0.12995760142803192, 0.1297539621

**Висновок:** бачимо, що точність досягає свого максимуму (88.07%) на `batch_size`=500. При цьому значенні `batch_size`, мінімуму сягає середньоквадратичне відхилення (RMSE). Також бачимо, що зі збільшенням `batch_size`, loss (відстань між передбаченими та реальними значеннями) помітно зменшується з 40.44 до 33.49. Причому, час навчання суттєво зменшується (з 182.67 секунд до майже 15.99 секунд). Тому, в якості найкращого значення `batch_size` оберемо **500**.


*Зауважимо, що описані ефекти відбуваються через те, що при більших значеннях batch_size відбувається наступне: градієнти стають більш стабільними, зменшується шум у градієнтах, покращується узагальнення (в деяких випадках), збільшується ефективність виконання навчання на CPU.*

### №6 Навчання на кращих гіперпараметрах та перевірка на перенавчання

Отже, було обрано такі найкращі гіперпараметри:

In [121]:
best_input_layer_neurons_number = 400
best_first_hide_layer_neurons_number = 200
best_second_hide_layer_neurons_number = 300
best_epochs_number = 15
best_batch_size = 500

Визначимо функцію, що створює модель *із найкращими параметрами*.

In [122]:
def create_basic_model_with_best_hyperparams():
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(best_input_layer_neurons_number, input_dim=784, activation='relu')) # relu(x)=max(0,x)
  model.add(L.Dense(best_first_hide_layer_neurons_number, activation='elu'))
  model.add(L.Dense(best_second_hide_layer_neurons_number, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy', 'RootMeanSquaredError'] # calculated using validation_data
  )

  return model

Створимо та натренуємо цю модель:

In [123]:
best_hyperparams_model = create_basic_model_with_best_hyperparams()


best_hyperparams_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 400)                 │         314,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 200)                 │          80,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 300)                 │          60,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           3,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 457,510 (1.75 MB)

 Trainable params: 457,510 (1.75 MB)

 Non-trainable params: 0 (0.00 B)

In [124]:
# train the model on best hyperparams
best_hyperparams_model.fit(
  x_train_float.reshape(-1, 28*28),
  y_train_oh,
  batch_size=best_batch_size, # Number of samples per gradient update.
  epochs=best_epochs_number,
  validation_data=(x_val_float.reshape(-1, 28*28), y_val_oh) # The model will not be trained on this data.
)

Epoch 1/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - RootMeanSquaredError: 0.1890 - accuracy: 0.7243 - loss: 0.7616 - val_RootMeanSquaredError: 0.1542 - val_accuracy: 0.8316 - val_loss: 0.4642
Epoch 2/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - RootMeanSquaredError: 0.1449 - accuracy: 0.8519 - loss: 0.4045 - val_RootMeanSquaredError: 0.1452 - val_accuracy: 0.8528 - val_loss: 0.4096
Epoch 3/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - RootMeanSquaredError: 0.1356 - accuracy: 0.8712 - loss: 0.3515 - val_RootMeanSquaredError: 0.1407 - val_accuracy: 0.8620 - val_loss: 0.3828
Epoch 4/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - RootMeanSquaredError: 0.1294 - accuracy: 0.8827 - loss: 0.3187 - val_RootMeanSquaredError: 0.1397 - val_accuracy: 0.8628 - val_loss: 0.3751
Epoch 5/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - RootMeanSquaredError: 0.1251 - accuracy: 0.8913 - loss: 0.2968 - val_RootMeanSquaredError: 0.1388 - val_accuracy: 0.8639 - val_loss: 0.3684
Epoch 6/15
120/120 ━━━━━━

Проведемо **крос-валідацію** моделі з найкращими гіперпараметрами

In [125]:
best_model_losses, best_model_accuracies, best_model_rmses, best_model_train_times = cross_validate(
  x_train_float, y_train_oh,
  lambda: create_basic_model_with_best_hyperparams(),
  batch_size=best_batch_size,
  number_of_epochs=best_epochs_number)

display_cross_validation_results(best_model_losses, best_model_accuracies, best_model_rmses, best_model_train_times)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - RootMeanSquaredError: 0.1320 - accuracy: 0.8821 - loss: 0.3600
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - RootMeanSquaredError: 0.1288 - accuracy: 0.8886 - loss: 0.3524
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - RootMeanSquaredError: 0.1299 - accuracy: 0.8847 - loss: 0.3451
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - RootMeanSquaredError: 0.1312 - accuracy: 0.8845 - loss: 0.3685
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - RootMeanSquaredError: 0.1350 - accuracy: 0.8756 - loss: 0.3829


Losses:  [0.36562520265579224, 0.35139909386634827, 0.3371163606643677, 0.3692191243171692, 0.3856426179409027]
Mean loss 0.361800479888916
Standard deviation of losses: 0.018409283727025407

Accuracies:  [0.8802499771118164, 0.8899999856948853, 0.8880833387374878, 0.8838333487510681, 0.8760833144187927]
Mean accuracy 0.8836499929428101
Standard deviation of accuracy: 0.005675061977103176

RMSEs:  [0.13250817358493805, 0.12839210033416748, 0.12829728424549103, 0.132169

**Висновок про перенавчання**

Як бачимо з результатів крос-валідації, модель є досить точною (точність 88.36%).

Проте, можна побачити, що під час навчання, похибка (RMSE) на валідаційних даних зменшується разом із похибкою на тренувальних даних ВКЛЮЧНО до 6-ї епохи. Те саме із функцією втрат. Аналогічно, точність зростає лише по 6-ту епоху. Починаючи з сьомої епохи, значення згаданих показників для валідаційних даних то зростають, то спадають. З цього можна зробити висновок, що в моделі присутнє перенавчання, починаючи із сьомої епохи.

Але, зважаючи на те, що на останній епосі різниця показників для тренувальних та валідаційних даних невелика (для похибки = 0.0345, для точності = 0.0537), а також зважаючи на високу точність моделі, можна зробити висновок, що перенавчання невелике. Перенавчання могло виникунути через завелику кількість епох, чи завелику кількість шарів та нейронів.

Для цієї моделі **реально** позбутись чи запобігти перенавчання. Для цього можна зробити наступне:
- Зменшити кількість епох до шести
- Використати регуляризацію
- Збільшити кількість тренувальних даних
- Зменшити кількість шарів чи нейронів
- Застосувати раннє зупинення (зупинення навчання, коли точність на валідаційних даних починає зменшуватись)
- Зменшити швидкість навчання (Learning Rate).

### №7 Застосування моделі в робочому режимі

Використаємо модель в робочому режимі на деяких валідаційних даних:

**Приклад №1**

In [130]:
img1 = x_val[1]
img1

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  13,  67,   0,
          0,   0,   0,  50,  38,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   8, 120, 209, 226, 247, 237,
        255, 255, 255, 247, 238, 235, 172,  72,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0, 137, 239, 252, 243, 234, 229, 238,
        244, 246, 240, 230, 232, 239, 248, 251, 194,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 102, 255, 231, 228, 227, 228, 233, 230,
        230, 229, 228, 232, 232, 231, 227, 224, 252, 179,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 233, 241, 229, 231, 255, 255, 238, 231,
        227, 238, 246, 228, 230, 227, 234, 235, 229, 241,  20,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 248, 241, 231, 255, 149,  47, 252, 228,
        255, 242, 216, 238, 232, 255, 228, 220, 234, 250,  54,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 255, 240, 232, 255,  15,   0, 255, 237,
        191,   0,   0, 214, 255,  13, 123, 255, 234, 252, 114,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   6, 255, 238, 239, 255, 177,   0, 255, 255,
          0, 130, 116,  47,  65,  43,  37, 255, 236, 249, 162,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  32, 255, 236, 245, 255, 204,   0, 255,  84,
          0,  37,  28,  31,   0,  25,  13, 255, 236, 249, 199,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  53, 255, 236, 250, 250, 231,   2, 255,  21,
          0, 221, 255, 236,  54, 245, 198, 243, 238, 245, 223,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  80, 255, 237, 250, 240, 255,   0,   0,  39,
        157,   0,   0, 215,  94,  20, 126, 255, 237, 239, 250,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 101, 255, 235, 253, 244, 243, 133, 138, 208,
        255, 201, 214, 255, 230,   7, 174, 255, 240, 238, 255,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 126, 255, 233, 255, 248, 233, 255, 255, 240,
        232, 243, 243, 231, 251, 255, 255, 254, 243, 238, 255,   3,   0,
          0,   0],
       [  0,   0,   0,   0, 147, 255, 233, 249, 181, 243, 227, 224, 230,
        234, 230, 230, 235, 228, 235, 222, 207, 255, 236, 255,  35,   0,
          0,   0],
       [  0,   0,   0,   0, 163, 255, 245, 221,  86, 255, 233, 233, 235,
        236, 234, 234, 234, 232, 242, 231, 125, 255, 236, 255,  55,   0,
          0,   0],
       [  0,   0,   0,   0, 181, 254, 255, 200,  69, 255, 228, 232, 234,
        235, 234, 234, 233, 235, 241, 237,  70, 255, 235, 246,  57,   0,
          0,   0],
       [  0,   0,   0,   0, 197, 247, 255, 188, 110, 255, 224, 233, 234,
        234, 234, 234, 234, 234, 240, 253,  69, 255, 236, 248,  77,   0,
          0,   0],
       [  0,   0,   0,   0, 200, 246, 255, 149, 145, 255, 223, 235, 234,
        235, 235, 235, 234, 237, 233, 255,  47, 255, 239, 249,  98,   0,
          0,   0],
       [  0,   0,   0,   0, 204, 243, 255, 111, 173, 255, 227, 235, 235,
        236, 235, 235, 235, 239, 229, 255,  19, 227, 246, 249, 110,   0,
          0,   0],
       [  0,   0,   0,   0, 196, 240, 255, 109, 213, 250, 229, 235, 235,
        236, 235, 237, 236, 237, 226, 255,  55, 203, 251, 245, 120,   0,
          0,   0],
       [  0,   0,   0,   0, 192, 243, 255, 114, 232, 240, 232, 235, 235,
        236, 234, 237, 236, 235, 229, 255, 134, 171, 252, 244, 137,   0,
          0,   0],
       [  0,   0,   0,   0, 189, 251, 255, 154, 238, 233, 236, 234, 235,
        236, 235, 238, 236, 235, 232, 255, 166, 125, 255, 243, 142,   0,
          0,   0],
       [  0,   0,   0,   0, 183, 252, 255, 171, 247, 232, 234, 234, 233,
        233, 232, 234, 233, 234, 233, 240, 223, 128, 255, 242, 151,   0,
          0,   0],
       [  0,   0,   0,   0, 178, 243, 255,  57, 238, 241, 238, 238, 238,
        237, 236, 237, 237, 240, 237, 254, 176,  52, 255, 239, 157,   0,
          0,   0],
       [  0,   0,   0,   0, 188, 240, 25

In [131]:
prediction1 = best_hyperparams_model.predict(x_val_float.reshape(-1, 28*28)[1:2])
print(prediction1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
[[7.2191074e-04 2.9910223e-09 9.9841881e-01 2.9324212e-09 3.3872147e-04
  4.1811216e-10 5.2062183e-04 7.2458439e-10 3.0551894e-10 7.7331802e-10]]


Отримали, що ця річ належить до категорії 2 (пуловер) з ймовірністю `99.98%`. Це відповідає дійсності, бо річ, що зображена на картинці, є пуловером.

**Приклад №2**

In [132]:
img2 = x_val[2]
img2

array([[  0,   0,   0,   0,   0,   0,   0,   0,   1,   0,  67, 177, 129,
        153, 117, 129, 146, 141, 175,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   2,   0, 196, 255, 255,
        255, 241, 251, 255, 245, 255,  76,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 243, 237, 217,
        228, 231, 222, 218, 219, 255, 116,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 235, 231,
        233, 226, 225, 229, 222, 255, 145,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  23, 255, 230, 229,
        233, 229, 225, 227, 221, 241, 167,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  67, 255, 225, 234,
        231, 226, 227, 228, 224, 234, 187,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 114, 249, 217, 235,
        238, 245, 227, 227, 225, 241, 176,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 167, 247, 227, 231,
        250, 200, 248, 231, 224, 251, 147,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 208, 242, 225, 243,
        225,   0, 255, 233, 226, 255, 129,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 223, 238, 223, 255,
        161,   0, 255, 236, 223, 255, 135,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 245, 235, 222, 255,
         72,   0, 248, 254, 217, 255, 145,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 233, 226, 255,
          7,   0, 191, 248, 220, 255, 126,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 233, 231, 251,
          0,   0, 159, 251, 221, 255, 111,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 232, 236, 216,
          0,   0, 131, 253, 220, 255, 111,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 228, 243, 172,
          0,   0,  57, 249, 222, 255, 111,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 230, 244, 139,
          0,   0,  10, 255, 227, 255, 100,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 227, 237, 242, 113,
          0,   0,   6, 255, 230, 255,  89,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 204, 239, 245, 151,
          0,   0,   0, 255, 227, 255,  99,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 151, 245, 238, 198,
          0,   0,   0, 255, 234, 255,  92,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  89, 248, 233, 248,
          0,   0,   0, 252, 237, 255,  90,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  31, 240, 229, 226,
          0,   0,   0, 239, 238, 255,  92,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 226, 233, 237,
         34,   0,   0, 234, 238, 255,  95,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 233, 236, 244,
         68,   0,   0, 209, 241, 237,  98,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   1,   0, 152, 242, 245,
        128,   0,   0, 198, 241, 238,  81,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   

In [133]:
prediction2 = best_hyperparams_model.predict(x_val_float.reshape(-1, 28*28)[2:3])
print(prediction2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[2.5403968e-10 1.0000000e+00 3.1372656e-13 2.0168267e-10 4.1134587e-12
  2.8190281e-15 6.7048239e-13 1.6468292e-15 2.6925651e-14 6.2949334e-14]]


Отримали, що ця річ належить до категорії 1 (брюки) з ймовірністю `100%`. Це відповідає дійсності, бо річ, що зображена на картинці, є брюками.

**Приклад №3**

In [134]:
img3 = x_val[10]
img3

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 145, 190,
         97, 109, 105, 187,  76,   0,   0,   0,   2,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  64, 170, 163, 208,
        243, 240, 229, 173, 162, 154,  18,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  39, 158, 196, 149, 191, 199,
        223, 243, 186, 191, 170, 155, 162, 126,  22,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   3,   0, 105, 161, 150, 141, 130, 199, 239,
        229, 220, 188, 244, 145, 128, 142, 151, 175,  23,   0,   2,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0, 150, 137, 134, 130, 147, 121, 241,
        255, 114, 240, 173, 142, 136, 132, 128, 153, 113,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  17, 157, 129, 146, 149, 137, 141, 177,
        200, 200, 207, 134, 145, 142, 140, 132, 129, 140,   3,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  50, 158, 137, 153, 150, 125, 155, 191,
        175, 141, 165, 166, 153, 130, 129, 132, 141, 149,  18,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  77, 166, 161, 137, 151, 166, 181, 216,
        182, 158, 215, 195, 155, 159, 162, 173, 150, 150,  58,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 104, 159, 170, 140, 132, 182, 198, 208,
        175, 136, 174, 169, 147, 155, 121, 188, 162, 142, 101,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 112, 171, 206, 179, 147, 163, 177, 196,
        192, 146, 187, 153, 142, 144, 136, 245, 202, 132, 137,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 126, 181, 219, 146, 157, 128, 121, 175,
        196, 104, 157, 158, 151, 140, 103,  89, 221, 141, 153,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 157, 175, 216,  55, 175, 153, 151, 188,
        214, 126, 204, 159, 145, 145, 116,  71, 212, 154, 155,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 183, 163, 232,  64, 138, 142, 182, 195,
        200, 158, 231, 146, 130, 161, 132, 105, 225, 146, 157,  10,   0,
          0,   0],
       [  0,   0,   0,   0,   6, 188, 161, 221,  54, 149, 165, 151, 167,
        188, 170, 181, 154, 141, 130, 144,  93, 207, 166, 142,  34,   0,
          0,   0],
       [  0,   0,   0,   0,  42, 182, 165, 236,  29, 146, 170, 141, 166,
        208, 124, 162, 173, 154, 132, 140,  97, 194, 153, 161,  47,   0,
          0,   0],
       [  0,   0,   0,   0,  58, 175, 171, 191,   5, 173, 162, 154, 181,
        207, 100, 161, 179, 155, 146, 149,  76, 192, 162, 150,  62,   0,
          0,   0],
       [  0,   0,   0,   0,  80, 179, 166, 167,  10, 182, 151, 157, 178,
        184, 166, 208, 174, 138, 146, 140,  73, 186, 161, 155,  71,   0,
          0,   0],
       [  0,   0,   0,   0, 100, 165, 171, 138,  51, 194, 157, 133, 187,
        196, 166, 198, 163, 129, 142, 134,  73, 191, 154, 155,  79,   0,
          0,   0],
       [  0,   0,   0,   0, 109, 158, 175, 118,  73, 179, 146, 149, 183,
        207, 141, 184, 171, 153, 137, 147,  73, 195, 157, 154,  85,   0,
          0,   0],
       [  0,   0,   0,   0, 109, 163, 179, 122, 116, 177, 144, 165, 182,
        190, 147, 179, 162, 146, 133, 153,  97, 161, 158, 150,  95,   0,
          0,   0],
       [  0,   0,   0,   0, 107, 158, 167, 163, 105, 137, 161, 163, 203,
        199, 149, 181, 167, 161, 147, 147,  99, 166, 157, 141,  99,   0,
          0,   0],
       [  0,   0,   0,   0, 109, 154, 166, 145,   0,  34,  84,  63, 107,
        149, 132, 153, 129, 124, 149,  56,   0, 114, 159, 141,  97,   0,
          0,   0],
       [  0,   0,   0,   0, 108, 154, 169,  85,   0,   0,   0,   0,   0,
          0,   2,   9,   0,   0,   0,   0,   0,  93, 157, 142,  99,   0,
          0,   0],
       [  0,   0,   0,   0, 101, 153, 169,  80,   0,   3,   0,   1,   0,
          0,   0,   0,   0,   0,   0,   1,   0,  80, 155, 144, 105,   0,
          0,   0],
       [  0,   0,   0,   0, 121, 157, 16

In [135]:
prediction3 = best_hyperparams_model.predict(x_val_float.reshape(-1, 28*28)[10:11])
print(prediction3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[7.8415195e-04 2.6558640e-05 1.1376380e-02 2.0866253e-06 9.2860162e-01
  2.0658824e-06 5.9139751e-02 2.1236601e-05 5.1462607e-06 4.0992280e-05]]


Отримали, що ця річ належить до категорії 4 (пальто) з ймовірністю `92.8%`. Це відповідає дійсності, бо річ, що зображена на картинці, є пальто.